# Multivariate Time Series Data PreProcessing <br> [[YouTube]](https://www.youtube.com/watch?v=jR0phoeXjrc)<br>

## AND<br>

# Multivariate Forecasting with LSTM <br> [[YouTube]](https://www.youtube.com/watch?v=ODEGJ_kh2aA)
<br>

In [1]:
import pandas as pd
from pylab import rcParams
import numpy as np
import seaborn as sns
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as ag
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline
from helpers import *
import tqdm as tq
from collections import defaultdict

In [2]:
sns.set(style='whitegrid', 
		palette='muted', 
		font_scale=1.2)

HAPPY_COLORS = ['#01BEFE', '#FFDD00', '#FF7D00', '#FF006D', 
				'#ADFF02', '#8F00FF']

sns.set_palette(sns.color_palette(HAPPY_COLORS))

rcParams['figure.figsize'] = 13, 7

pl.seed_everything(123)
import yfinance as yf
tq.tqdm.pandas()

Global seed set to 123


<font size = 4><span style = 'background-color: #ddddff; padding: 5px 5px 3px 5px; line-height: 1.5; color:black;border-radius: 3px;'>**compile_stocks()**
* Yahoo Finance only allows 7 days worth of data at 1 minute intervals
* This function will compile dataframes in 7 day increments for an entire range of dates
* They also only allow 1 min data for up to 30 days prior
* This can be used for other intervals that allow for longer time periods

In [3]:
def compile_stocks(symbol, end, start, day_window, interval):
	import datetime
	import yfinance as yf
	
	end_date = end
	start_date = (pd.to_datetime(end) - datetime.timedelta(days = day_window))
	
	dfs = []
	stop_me = False
	
	while pd.to_datetime(start_date) >= pd.to_datetime(start):
		df = yf.download(symbol, 
						 start = start_date,
						 end = end_date, 
						 interval = interval)

		dfs.append(df)
		end_date = start_date
		start_date = start_date - datetime.timedelta(days = day_window)

		if start_date < pd.to_datetime(start):
			start_date = pd.to_datetime(start)
		else:
			start_date = start_date

		if start_date == end_date:
			break
		
	master_df = pd.concat(dfs).sort_values(by="Datetime")
		
	return master_df

In [4]:
data = compile_stocks('BTC-USD', 
					 '2023-01-22', 
					 '2022-12-25',
					 7, '1m')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


<font size = 4><span style = 'background-color: #ddddff; padding: 5px 5px 3px 5px; line-height: 1.5; color:black;border-radius: 3px;'>**The Data**

In [5]:
df = data.copy()
df.columns = [x.lower() for x in df.columns]
head_tail_vert(df, 3, 'raw bitcoin data', intraday=True)

,open,high,low,close,adj close,volume
Datetime,,,,,,
2022-12-25 00:00:00+00:00,"16,847.51","16,847.51","16,847.51","16,847.51","16,847.51",0
2022-12-25 00:01:00+00:00,"16,846.72","16,846.72","16,846.72","16,846.72","16,846.72",0
2022-12-25 00:02:00+00:00,"16,846.70","16,846.70","16,846.70","16,846.70","16,846.70",0


,open,high,low,close,adj close,volume
Datetime,,,,,,
2023-01-21 23:56:00+00:00,"22,780.84","22,780.84","22,780.84","22,780.84","22,780.84",0
2023-01-21 23:57:00+00:00,"22,784.79","22,784.79","22,784.79","22,784.79","22,784.79",0
2023-01-21 23:58:00+00:00,"22,777.93","22,777.93","22,777.93","22,777.93","22,777.93",10856448


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 40074 entries, 2022-12-25 00:00:00+00:00 to 2023-01-21 23:58:00+00:00
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   open       40074 non-null  float64
 1   high       40074 non-null  float64
 2   low        40074 non-null  float64
 3   close      40074 non-null  float64
 4   adj close  40074 non-null  float64
 5   volume     40074 non-null  int64  
dtypes: float64(5), int64(1)
memory usage: 2.1 MB


In [7]:
df.shape

(40074, 6)

<font size = 4><span style = 'background-color: #ddddff; padding: 5px 5px 3px 5px; line-height: 1.5; color:black;border-radius: 3px;'>**Shift Method**
* using `shift()` to add the previous timestamp value
* then creating a difference column to show change since last
* creating a function to do all this

In [8]:
def add_change_column(df, column_changing, new_col_name):
	df['previous'] = df[column_changing].shift()
	df = df.drop(df.index[0])
	df[new_col_name] = df[column_changing] - df.previous
	df = df.drop(columns = ['previous'])
	return df

In [9]:
df = add_change_column(df, 'close', 'change')

In [15]:
head_tail_vert(df, 3, "Change column added", intraday=True)

,open,high,low,close,adj close,volume,change
Datetime,,,,,,,
2022-12-25 00:01:00+00:00,"16,846.72","16,846.72","16,846.72","16,846.72","16,846.72",0,-0.79
2022-12-25 00:02:00+00:00,"16,846.70","16,846.70","16,846.70","16,846.70","16,846.70",0,-0.02
2022-12-25 00:03:00+00:00,"16,846.54","16,846.54","16,846.54","16,846.54","16,846.54",0,-0.17


,open,high,low,close,adj close,volume,change
Datetime,,,,,,,
2023-01-21 23:56:00+00:00,"22,780.84","22,780.84","22,780.84","22,780.84","22,780.84",0,3.23
2023-01-21 23:57:00+00:00,"22,784.79","22,784.79","22,784.79","22,784.79","22,784.79",0,3.95
2023-01-21 23:58:00+00:00,"22,777.93","22,777.93","22,777.93","22,777.93","22,777.93",10856448,-6.87


<font size = 4><span style = 'background-color: #ddddff; padding: 5px 5px 3px 5px; line-height: 1.5; color:black;border-radius: 3px;'>**Feature Engineering**

<font size = 4><b> The following is Valkov's (tutorial guide) method of adding these features, which loops through the entire dataframe. This makes no sense in Pandas. 

In [16]:
def his_featurize(df):
	rows = []
	df['date'] = df.index
	
	for item, row in df.iterrows():
		row_data = dict(
			weekday = row.date.dayofweek,
			month_day = row.date.day,
			year_week = row.date.week,
			month = row.date.month,
			open = row.open,
			high = row.high,
			low = row.low,
			close = row.close,
			change = row.change)

		rows.append(row_data)
		
	features_df = pd.DataFrame(rows)
	
	return features_df

<font size = 4><b> This is my version. Much better.

In [17]:
def featurize_df(df):
	df['weekday'] = df.index.dayofweek
	df['month_day'] = df.index.day
	df['year_week'] = df.index.isocalendar().week
	df['month'] = df.index.month
	
	return df

In [18]:
%%time
test = his_featurize(df.copy())

CPU times: user 2.23 s, sys: 24.7 ms, total: 2.26 s
Wall time: 2.27 s


In [19]:
%%time
df = featurize_df(df)

CPU times: user 30.3 ms, sys: 2.41 ms, total: 32.7 ms
Wall time: 30.6 ms


In [20]:
head_tail_vert(df, 3, 'df with added features')

,open,high,low,close,adj close,volume,change,weekday,month_day,year_week,month
Datetime,,,,,,,,,,,
2022-12-25,"16,846.72","16,846.72","16,846.72","16,846.72","16,846.72",0,-0.79,6,25,51,12
2022-12-25,"16,846.70","16,846.70","16,846.70","16,846.70","16,846.70",0,-0.02,6,25,51,12
2022-12-25,"16,846.54","16,846.54","16,846.54","16,846.54","16,846.54",0,-0.17,6,25,51,12


,open,high,low,close,adj close,volume,change,weekday,month_day,year_week,month
Datetime,,,,,,,,,,,
2023-01-21,"22,780.84","22,780.84","22,780.84","22,780.84","22,780.84",0,3.23,5,21,3,1
2023-01-21,"22,784.79","22,784.79","22,784.79","22,784.79","22,784.79",0,3.95,5,21,3,1
2023-01-21,"22,777.93","22,777.93","22,777.93","22,777.93","22,777.93",10856448,-6.87,5,21,3,1


In [21]:
describe_em(df, ['close', 'volume', 'change'])

,close
count,"40,073.00"
mean,"18,235.33"
std,"2,011.31"
min,"16,408.47"
25%,"16,735.81"
50%,"16,944.59"
75%,"20,756.78"
max,"23,282.35"
,volume
count,"40,073.00"


In [22]:
df.shape

(40073, 11)

<font size = 4><span style = 'background-color: #ddddff; padding: 5px 5px 3px 5px; line-height: 1.5; color:black;border-radius: 3px;'>**Splitting the Data**

In [23]:
train_size = int(len(df) * .8)
pretty(f'number of training inputs: {train_size:,}.')
test_size = int(len(df) * .2)
pretty(f'number of testing inputs: {test_size:,}.')

In [24]:
train_df, test_df = df[:train_size], df[train_size + 1:]

In [25]:
pretty(f'train_df.shape: {train_df.shape}  |  test_df.shape: {test_df.shape}')

<font size = 4><span style = 'background-color: #ddddff; padding: 5px 5px 3px 5px; line-height: 1.5; color:black;border-radius: 3px;'>**Scaling the data**

In [26]:
scaler = MinMaxScaler(feature_range = (-1, 1))
scaler = scaler.fit(train_df)

In [27]:
train_df = pd.DataFrame(scaler.transform(train_df), 
										  index = train_df.index, 
										  columns = train_df.columns)

test_df = pd.DataFrame(scaler.transform(test_df), 
										 index = test_df.index, 
										 columns = test_df.columns)

<font size = 4><span style = 'background-color: #ddddff; padding: 5px 5px 3px 5px; line-height: 1.5; color:black;border-radius: 3px;'>**Viewing the scaled data**

In [29]:
multi([(train_df.head(5),'train_df.head(5)'),
	   (train_df.tail(5),'train_df.tail(5)'),
	   (test_df.head(5),'test_df.head(5)'),
	   (test_df.tail(5),'test_df.tail(5)')], intraday=True)

,open,high,low,close,adj close,volume,change,weekday,month_day,year_week,month
Datetime,,,,,,,,,,,
2022-12-25 00:01:00+00:00,-0.82,-0.82,-0.82,-0.82,-0.82,-1.00,-0.22,1.00,0.60,0.96,1.00
2022-12-25 00:02:00+00:00,-0.82,-0.82,-0.82,-0.82,-0.82,-1.00,-0.22,1.00,0.60,0.96,1.00
2022-12-25 00:03:00+00:00,-0.82,-0.82,-0.82,-0.82,-0.82,-1.00,-0.22,1.00,0.60,0.96,1.00
2022-12-25 00:04:00+00:00,-0.82,-0.82,-0.82,-0.82,-0.82,-1.00,-0.22,1.00,0.60,0.96,1.00
2022-12-25 00:05:00+00:00,-0.82,-0.82,-0.82,-0.82,-0.82,-1.00,-0.22,1.00,0.60,0.96,1.00
,open,high,low,close,adj close,volume,change,weekday,month_day,year_week,month
Datetime,,,,,,,,,,,
2023-01-16 09:16:00+00:00,0.78,0.78,0.78,0.78,0.78,-0.97,-0.17,-1.00,0.00,-0.92,-1.00
2023-01-16 09:17:00+00:00,0.78,0.78,0.78,0.78,0.78,-0.99,-0.22,-1.00,0.00,-0.92,-1.00


<font size = 4><span style = 'background-color: #ddddff; padding: 5px 5px 3px 5px; line-height: 1.5; color:black;border-radius: 3px;'>**Creating sequences**

<font size = 4><span style = 'background-color: #ddddff; padding: 5px 5px 3px 5px; line-height: 1.5; color:black;border-radius: 3px;'>**Text**

<font size = 4><span style = 'background-color: #ddddff; padding: 5px 5px 3px 5px; line-height: 1.5; color:black;border-radius: 3px;'>**Text**

<font size = 4><span style = 'background-color: #ddddff; padding: 5px 5px 3px 5px; line-height: 1.5; color:black;border-radius: 3px;'>**Text**

<font size = 4><span style = 'background-color: #ddddff; padding: 5px 5px 3px 5px; line-height: 1.5; color:black;border-radius: 3px;'>**Text**